In [ ]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [ ]:
from posarmctools.ekfnavtools import *
%load_ext autoreload
%autoreload 2

In [ ]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

## Load navigation data SbgLogEkfNav

In [ ]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/dataLogger/session_0000/2017_10_10/09h00/"

logEkfEuler = prefix + "sbgLogEkfEuler.dat"
logEkfNav  = prefix + "sbgLogEkfNav.dat"
logUtcData = prefix + "sbgLogUtcData.dat"

Load logEkfNav data 09h00

Take off at 11h42

In [ ]:
# logEkfNav
timeStamps = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [ ]:
Vel = (vN**2 + vE**2 + vD**2)**0.5

### Plot navigation data jointly with track references

In [ ]:
validNav = np.where(Long < 0)

In [ ]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )

plt.title("Navigation data and track references 2017_10_10 09h00")

### Check the existence of record files

In [ ]:
firstRecord = 0
lastRecord = 336
aux = np.zeros(3372)
fileList = []
dataDir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_11_49_09"
for k in range(firstRecord, lastRecord, blocksPerFile):
    filename = dataDir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)
idxFileList = np.arange( len( fileList ) ) * blocksPerFile + firstRecord

In [ ]:
plt.plot(idxFileList, fileList, 'o-')

In [ ]:
# times recorded by Windows XP
t_0    = (11, 49, 9)

record0Start = 0
record0Stop = 336

Look for the time data corresponding to the records

In [ ]:
hourOffset = 2

#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8
utc_timeStamp, utc_h, utc_m, utc_s, utc_nano = np.loadtxt( logUtcData, skiprows = 1, 
                                           usecols = (0, idx_h, idx_m, idx_s, idx_nano), unpack=True )
utc_seconds = (utc_h + hourOffset) * 3600 + utc_m * 60 + utc_s + utc_nano * 1e-9

In [ ]:
# compute time of day using the date of block 0 and a computed delta

record0_tStart = getTimeOfDay( getSeconds( t_0 ) + record0Start / blocksPerFile * T_files )
record0_tStop = getTimeOfDay( getSeconds( t_0 ) + record0Stop / blocksPerFile * T_files )

In [ ]:
record0_idx = getIndices (record0_tStart, record0_tStop, utc_seconds)

### Process timeStamps.data

In [ ]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_11_49_09"
timeStampsFile = data_dir + "/2017_10_10_11_49_09_timeStamps.data"
blockNumber, timeStamp = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [ ]:
timeStampStart = timeStamp[0]
timeStampStop = timeStamp[-1]

In [ ]:
idxStart = np.where(utc_timeStamp <= timeStampStart)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop)[0][-1]
record0_idx_alt = (idxStart, idxStop)

### Plot navigation data and record periods

In [ ]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )
plotRunaway( ax )

#addOnPlot( ax, Long, Lat, record0_idx, 'yellow', 'record 0 (time line dated by the MEB)' )
addOnPlot( ax, Long, Lat, record0_idx_alt, 'orange', 'record 11_49_09 (time line dated by SBG time stamps)' )

plt.title("Navigation data, track references, record locations\nFlight 2017_10_10_09h00")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Track 1

In [ ]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * track1_Long_0)
ax1.plot(np.ones(Long.shape) * track1_Long_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * track1_Lat_0)
ax2.plot(np.ones(Lat.shape) * track1_Lat_1)

### Plot attitude data

In [ ]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [ ]:
euler_timeStamp, roll, pitch, yaw = np.loadtxt( logEkfEuler, skiprows = 1, 
                                               usecols = (0, idx_roll, idx_pitch, idx_yaw), unpack=True )

In [ ]:
idxRange = range(idxStart, idxStop)

plt.figure()

plt.subplot(211)
plt.title("Altitude and velocity during record 11_49_09")
plt.plot( Alt[idxRange], label="altitude" )
plt.legend()
plt.grid()
plt.subplot(212)
plt.plot( Vel[idxRange], label="velocity" )
plt.legend()
plt.grid()

In [ ]:
plt.figure()

plt.subplot(311)
plt.title("Pitch roll and yaw during record record 11_49_09")
plt.plot( pitch[idxRange], label="pitch" )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( roll[idxRange], label="roll" )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( yaw[idxRange], label="yaw" )
plt.legend()
plt.grid()